Le **web scraping** consiste à extraire des données de sites web. Selenium Testing est un outil populaire pour le web scraping, car il peut interagir avec les pages web comme un humain, ce qui le rend particulièrement utile pour extraire du contenu dynamique généré par JavaScript.
**Etapes pour travailler avec web sraping**

1.  Installer Selenium :pip install selenium
2.  Téléchargez WebDriver :
      *  **ChromeDriver** : Télécharger ChromeDriver
      *  **GeckoDriver (Firefox)** : Télécharger GeckoDriver

**Selenium **est compatible avec plusieurs navigateurs, mais nous utiliserons **Google Chrome**, le plus populaire. Vous devrez également installer les éléments suivants :

**Google Chrome** : La dernière version du navigateur fera l'affaire.
**ChromeDriver** : Téléchargez celui qui correspond à votre version de Chrome*(pour savoir la version de chrome tape ceci *chrome://settings/help* dans chrome)*, extrayez le dossier zippé et collez le fichier exécutable ( chromedriver.exe) dans le dossier racine de votre projet.


# Step #1: Build a Basic Scraper #
  Initialize a **scraper.py** file in your project directory . The code instantiates a ChromeDriver object and uses it to visit the target site before quitting the driver instance:

In [4]:
%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\naoui\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Importer des bibliothèques et configurer WebDriver : #

In [6]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.service import Service

import os

# Set th path for webdriver #

In [7]:
path='chromedriver-win64/chromedriver-win64/chromedriver.exe'

 Dans les nouvelles versions de *Selenium (v4+)*, le paramètre executable_path n'existe plus dans le constructeur de *webdriver.Chrome()*.

Depuis *Selenium 4*, il faut utiliser la classe Service pour spécifier le chemin du driver.

In [8]:
#creer un service
service = Service(executable_path=path)
#open the browser
browser = webdriver.Chrome(service=service)

In [10]:
#chargement de page(visit the target site)
browser.get('https://books.toscrape.com')
# Affichage complet de la page html
#print(browser.page_source)


In [12]:
#maximize the window
browser.maximize_window()

#Étape 2 : Configurer le mode headless #

Activez le mode headless pour Chrome dans Selenium en définissant un ChromeOptionsobjet et en le transmettant au constructeur WebDriver de Chrome. De plus, vous devez définir headless=new pour activer le mode headless 

In [11]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions()

# set the options to use Chrome in headless mode
options.add_argument("--headless=new")
 
# initialize an instance of the Chrome driver (browser) in headless mode
browser = webdriver.Chrome(options=options)

# Étape 3 : Extraire des données spécifiques de la page #

Il existe deux méthodes pour localiser des éléments sur une page Web :
*find_element:* Il ne trouve qu'un seul élément. Si plusieurs éléments partagent des sélecteurs similaires, il renvoie le premier élément HTML correspondant à la condition de recherche.
*find_elements:* Il renvoie tous les éléments qui correspondent à la condition de recherche dans un tableau.
Ces deux méthodes prennent en charge huit stratégies de localisation. Nous pouvons les classer en trois catégories :
*Sélecteurs CSS : By.ID , By.CLASS_NAME, et By.CSS_SELECTOR*.
*XPath : By.XPATH* .
*Sélecteurs directs : By.NAME , By.LINK_TEXT, By.PARTIAL_LINK_TEXT, et By.TAG_NAME*.

In [13]:
from selenium.webdriver.common.by import By
import pandas as pd

In [14]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# === LANCEMENT SELENIUM ===
driver = webdriver.Chrome()
driver.get("https://books.toscrape.com/catalogue/page-1.html")

books = []

page_number = 1
while True:
    print(f" Scraping page {page_number}...")
    
    # Attendre que les livres soient chargés
    WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "product_pod"))
    )
    
    # Récupérer tous les liens des livres sur la page
    book_links = driver.find_elements(By.XPATH, "//h3/a")
    links = [link.get_attribute("href") for link in book_links]

    # Parcourir chaque livre
    for link in links:
        driver.get(link)

        # Titre
        title = driver.find_element(By.TAG_NAME, "h1").text

        # Description (parfois absente)
        try:
            description = driver.find_element(
                By.XPATH, "//div[@id='product_description']/following-sibling::p"
            ).text
        except:
            description = "Pas de description"

        # Prix
        price = driver.find_element(By.CLASS_NAME, "price_color").text

        # Disponibilité
        availability = driver.find_element(By.CLASS_NAME, "availability").text.strip()

        # Image
        image_url = driver.find_element(By.CSS_SELECTOR, ".item.active img").get_attribute("src")

        # Note
        rating_class = driver.find_element(By.CLASS_NAME, "star-rating").get_attribute("class").split()[-1]
        rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
        rating = rating_map.get(rating_class, None)

        books.append({
            "Title": title,
            "Description": description,
            "Price": price,
            "Availability": availability,
            "Image_URL": image_url,
            "Rating": rating
        })

    # Aller à la page suivante si possible
    try:
        next_button = driver.find_element(By.CLASS_NAME, "next").find_element(By.TAG_NAME, "a")
        next_page_url = next_button.get_attribute("href")
        driver.get(next_page_url)
        page_number += 1
    except:
        print(" Fin du scraping - plus de pages.")
        break



 Scraping page 1...
 Fin du scraping - plus de pages.


In [15]:
# Créer le DataFrame
df = pd.DataFrame(books)
print(df.head())

                                   Title  \
0                   A Light in the Attic   
1                     Tipping the Velvet   
2                             Soumission   
3                          Sharp Objects   
4  Sapiens: A Brief History of Humankind   

                                         Description   Price  \
0  It's hard to imagine a world without A Light i...  £51.77   
1  "Erotic and absorbing...Written with starling ...  £53.74   
2  Dans une France assez proche de la nôtre, un h...  £50.10   
3  WICKED above her hipbone, GIRL across her hear...  £47.82   
4  From a renowned historian comes a groundbreaki...  £54.23   

              Availability                                          Image_URL  \
0  In stock (22 available)  https://books.toscrape.com/media/cache/fe/72/f...   
1  In stock (20 available)  https://books.toscrape.com/media/cache/08/e9/0...   
2  In stock (20 available)  https://books.toscrape.com/media/cache/ee/cf/e...   
3  In stock (20 available)

In [ ]:
# Sauvegarder en CSV
df.to_csv("books_list.csv", index=False, encoding="utf-8")
print(" Données sauvegardées dans books_list.csv")


In [ ]:

# Récupérer l'élément <h1>
titles = browser.find_elements(By.XPATH, "//h3/a")
for t in titles:
    print(t.text)


A Light in the ...
Tipping the Velvet
Soumission
Sharp Objects
Sapiens: A Brief History ...
The Requiem Red
The Dirty Little Secrets ...
The Coming Woman: A ...
The Boys in the ...
The Black Maria
Starving Hearts (Triangular Trade ...
Shakespeare's Sonnets
Set Me Free
Scott Pilgrim's Precious Little ...
Rip it Up and ...
Our Band Could Be ...
Olio
Mesaerion: The Best Science ...
Libertarianism for Beginners
It's Only the Himalayas


In [20]:
from selenium.webdriver.common.by import By

# Titres
titles = browser.find_elements(By.XPATH, "//h3/a")

# Prix
prices = browser.find_elements(By.CLASS_NAME, "price_color")

# Stock (utilisation CSS selector pour classes multiples)
stock_availability = browser.find_elements(By.CSS_SELECTOR, ".instock.availability")

# Images (on récupère les <img> à l'intérieur des balises avec classe 'thumbnail')
images = browser.find_elements(By.CSS_SELECTOR, ".thumbnail img")

# Ratings
ratings = browser.find_elements(By.CLASS_NAME, "star-rating")

# Affichage
for i in range(len(titles)):
    title = titles[i].text
    price = prices[i].text
    stock = stock_availability[i].text.strip()
    image_url = images[i].get_attribute("src")
    rating_class = ratings[i].get_attribute("class").replace("star-rating", "").strip()

    print(f"Titre: {title}")
    print(f"Prix: {price}")
    print(f"Stock: {stock}")
    print(f"Image: {image_url}")
    print(f"Note: {rating_class}")
    print("------")


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=139.0.7258.67); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff76cc66b55+79621]
	GetHandleVerifier [0x0x7ff76cc66bb0+79712]
	(No symbol) [0x0x7ff76c9fc0ea]
	(No symbol) [0x0x7ff76c9e8355]
	(No symbol) [0x0x7ff76ca0cefa]
	(No symbol) [0x0x7ff76ca82845]
	(No symbol) [0x0x7ff76caa2d02]
	(No symbol) [0x0x7ff76ca7af13]
	(No symbol) [0x0x7ff76ca44151]
	(No symbol) [0x0x7ff76ca44ee3]
	GetHandleVerifier [0x0x7ff76cf2686d+2962461]
	GetHandleVerifier [0x0x7ff76cf20b8d+2938685]
	GetHandleVerifier [0x0x7ff76cf3f74d+3064573]
	GetHandleVerifier [0x0x7ff76cc80c9e+186446]
	GetHandleVerifier [0x0x7ff76cc88a6f+218655]
	GetHandleVerifier [0x0x7ff76cc6f944+115956]
	GetHandleVerifier [0x0x7ff7

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# --- Configurer Selenium ---
chrome_options = Options()
chrome_options.add_argument("--headless")  # Optionnel : ne pas afficher la fenêtre
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=chrome_options)

# --- Aller sur le site ---
browser.get("https://books.toscrape.com")

# --- Trouver tous les livres ---
book_links = browser.find_elements(By.XPATH, "//h3/a")

# On sauvegarde les liens pour pouvoir y revenir
book_urls = [link.get_attribute("href") for link in book_links]

# --- Boucle sur chaque livre ---
for url in book_urls:
    browser.get(url)
    time.sleep(1)  # laisser le temps de charger

    # Titre
    title = browser.find_element(By.TAG_NAME, "h1").text

    # Prix
    price = browser.find_element(By.CLASS_NAME, "price_color").text

    # Stock
    stock = browser.find_element(By.CSS_SELECTOR, ".instock.availability").text.strip()

    # Image
    image_url = browser.find_element(By.CSS_SELECTOR, ".thumbnail img").get_attribute("src")

    # Note (étoiles)
    rating_class = browser.find_element(By.CLASS_NAME, "star-rating").get_attribute("class")
    rating = rating_class.replace("star-rating", "").strip()

    # Description
    try:
        description = browser.find_element(By.CSS_SELECTOR, "#product_description + p").text
    except:
        description = "Pas de description disponible"

    # --- Affichage ---
    print(f"Titre: {title}")
    print(f"Prix: {price}")
    print(f"Stock: {stock}")
    print(f"Image: {image_url}")
    print(f"Note: {rating}")
    print(f"Description: {description}")
    print("-" * 50)

browser.quit()


In [ ]:
# Fermer le navigateur
browser.quit()